## About the ScorpioBroker context broker in the Smart Village Platform

> Why do some  people think reactive applications like an IoT Agent are lazy? Because they need to be commanded in all things. 

An IoT Agent receives messages from smart devices to update the device state, or perform some kind of action on the device. The IoT Agent is a reactive application that can register smart devices, connect to various types of message brokers, receive messages in a native device protocol, and send updates to a context broker following the NGSI-LD standard format for smart device data. You can find the full [NGSI-LD API Operation Definition in section 5 of this PDF]. NGSI-LD stores entities as the primary data stored. Entities can be any data representing any [Smart Device Model data](https://github.com/smart-data-models) in the world. There are so many Smart Data Models in the world, but they can be grouped into the following domains: Smart Cities, Smart Agrifood, Smart Water, Smart Energy,  Smart Logistics, Smart Robotics, Smart Sensoring, Cross sector, Smart Health, Smart Destination, Smart Environment, Smart Aeronautics, Smart Manufacturing, Incubated,  Harmonization. Don't feel limited to stick to these domains, you can create your own smart data models and follow the NGSI-LD specification to confirm to all the latest open standards for managing smart device data. Here are some of the native device protocols and corresponding IoT Agents that can connect the device to the context broker. 


- [IoTAgent-JSON](https://fiware-iotagent-json.readthedocs.io/en/latest/) - a bridge between HTTP/MQTT messaging (with a JSON payload) and NGSI
- [IoTAgent-LWM2M](https://fiware-iotagent-lwm2m.readthedocs.io/en/latest) - a bridge between the Lightweight M2M protocol and NGSI
- [IoTAgent-UL](https://fiware-iotagent-ul.readthedocs.io/en/latest) - a bridge between HTTP/MQTT messaging (with an UltraLight2.0 payload) and NGSI
- [IoTAgent-LoRaWAN](https://fiware-lorawan.readthedocs.io/en/latest) - a bridge between the LoRaWAN protocol and NGSI
- [IoTagent-OPCUA](https://iotagent-opcua.readthedocs.io/en/latest/) - a bridge between the OPC-UA protocol and NGSI


## Deploy the IoT Agent JSON in NERC OpenShift

If you are curious what the IoTAgentJson custom resource definition looks like that you are deploying in NERC OpenShift, run the command below. 

In [ ]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/iotagentjsons/iotagent-json/iotagentjson.yaml
echo DONE

Here are some useful things to note about the configuration of the IoT Agent JSON. 

- `ngsild:` We configure the host and port of the context broker, we use the ScorpioBroker host and port in the same namespace. 
- `mongodb:` We configure the name of MongoDB, and the kubernetes Secret name and key for the MongoDB password. This allows us to connect to MongoDB to store smart device registrations. 
- `message_broker:` We configure the host of the message broker, and the kubernetes Secret name and key for the RabbitMQ password. This allows us to connect to RabbitMQ and start listening for messages. 
- `replicas: 1` We'll only deploy 1 replica for NERC OpenShift to stay within the resource quota of NERC OpenShift. 
- `route`: We create a public route for the IoT Agent API if you want to interact with it from your own computer. 
- The `resources:` definition defines the memory and CPU requests and limits for our IoT Agent JSON pod. We have to keep this lower than I would recommend for a production deployment because we have limited resources available in NERC OpenShift for the many running Smart Village services. 

For more information about the IoTAgentJson custom resource definition, [see the full IoTAgentJson schema here](https://github.com/smartabyar-smartvillage/smartvillage-operator/blob/main/config/crd/bases/smartvillage.computate.org_iotagentjsons.yaml). 

Run the Ansible Playbook below to deploy IoT Agent JSON to the cloud. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-iotagentjson.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/iotagentjsons/iotagent-json/iotagentjson.yaml
echo DONE

### View IoT Agent pod details
After running the Ansible Playbook, it will take a minute before the IoT Agent pod is up and running. Run the command below until the IoT Agent pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=iotagent-json
oc -n smart-village-faeeb6c wait pod -l app.kubernetes.io/instance=iotagent-json --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=iotagent-json
echo DONE

### View IoT Agent JSON pod logs
If your IoT Agent JSON pod does not reach the STATUS Running, you can run the command below to view the pod logs of IoT Agent JSON and check for other errors that may have occured. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c logs -l app.kubernetes.io/instance=scorpiobroker
echo DONE

## Next...
I hope that answers your questions about IoT Agent JSON in the Smart Village Platform. 
- If you have additional questions or issues, please [create an issue for the course here](https://github.com/smartabyar-smartvillage/smartabyar-smartvillage-nerc-course/issues). 
- Otherwise, please continue to the next notebook [08-about-zookeeper.ipynb](08-about-zookeeper.ipynb). 